In [1]:
import cv2
import dlib
import numpy as np
import pyautogui as pag

def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((68, 2), dtype=dtype)
	# loop over the 68 facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
	# return the list of (x, y)-coordinates
	return coords

def eye_on_mask(mask, side):
    points = [shape[i] for i in side]
    points = np.array(points, dtype=np.int32)
    mask = cv2.fillConvexPoly(mask, points, 255)
    l = points[0][0]
    t = (points[1][1]+points[2][1])//2
    r = points[3][0]
    b = (points[4][1]+points[5][1])//2
    # print([l,t,r,b])
    # cv2.circle(img, (l,t), 4, (0, 0, 255), 2)
    # cv2.circle(img, (r,b), 4, (0, 0, 255), 2)
    return mask, [l, t, r, b]

def find_eyeball_position(end_points, cx, cy):
    try:
        x_ratio = (end_points[0] - cx)/(cx - end_points[2])
        y_ratio = (cy - end_points[1])/(end_points[3] - cy)
        # print(end_points[0])
        #print(x_ratio)
        if x_ratio > 3:
            return 1
        elif x_ratio < 0.33:
            return 2
        elif y_ratio < 0.33:
            return 3
        else:
            return 0
    except:
        pass

def contouring(thresh, mid, img, end_points, right=False):
    cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    try:
        cnt = max(cnts, key = cv2.contourArea)
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        # cv2.circle(img, (cx,cy), 4, (0, 0, 255), 2)
        # print(cx,cy)
        if right:
            cx += mid
        #cv2.circle(img, (cx, cy), 4, (0, 0, 255), 2)
        return find_eyeball_position(end_points, cx, cy)
    except:
        pass

def process_thresh(thresh):
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)
    thresh = cv2.medianBlur(thresh, 3)
    thresh = cv2.bitwise_not(thresh)
    return thresh

def print_eye_pos(left, right):
    if left == right and left != 0:
        text = ' '
        drag = 18
        if left == 1:
            pag.moveRel(-drag,0)
            print('Looking left')
            text = 'Looking left'
        elif left == 2:
            pag.moveRel(drag,0)
            print('Looking right')
            text = 'Looking right'
        elif left == 3:
            pag.moveRel(0,-drag)
            print('Looking up')
            text = 'looking up'
        elif left == 0:
            pag.moveRel(0, drag)
            print('Looking down')
            text = 'Looking down'

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, text, (30,30), font, 1, (0,255,255), 2, cv2.LINE_AA)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

modelFile = "C:\\Users\\ADMIN\\Documents\\final year project\\res10_300x300_ssd_iter_140000.caffemodel"
configFile = "C:\\Users\\ADMIN\\Documents\\final year project\\deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

left = [36, 37, 38, 39, 40, 41]
right = [42, 43, 44, 45, 46, 47]

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
ret, img = cap.read()
thresh = img.copy()

cv2.namedWindow('image')
kernel = np.ones((9, 9), np.uint8)

def nothing(x):
    pass
cv2.createTrackbar('threshold', 'image', 0, 255, nothing)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces3 = net.forward()
    for i in range(0, faces3.shape[2]):
        confidence = faces3[0, 0, i, 2]
        if confidence > 0.5:
            box = faces3[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            b = [x,y,x1,y1]
            # print(b)
            shape = predictor(gray,dlib.rectangle(x,y,x1,y1))
            shape = shape_to_np(shape)
    # rects = detector(gray, 1)
    # for rect in rects:
    #     print(rect)
    #     shape = predictor(gray, rect)
    #     shape = shape_to_np(shape)
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            mask, end_points_left = eye_on_mask(mask, left)
            mask, end_points_right = eye_on_mask(mask, right)
            mask = cv2.dilate(mask, kernel, 5)
            eyes = cv2.bitwise_and(img, img, mask=mask)
            mask = (eyes == [0, 0, 0]).all(axis=2)
            eyes[mask] = [255, 255, 255]
            mid = (shape[42][0] + shape[39][0]) // 2
            eyes_gray = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
            threshold = cv2.getTrackbarPos('threshold', 'image')
            _, thresh = cv2.threshold(eyes_gray, threshold, 255, cv2.THRESH_BINARY)
            thresh = process_thresh(thresh)
            eyeball_pos_left = contouring(thresh[:, 0:mid], mid, img, end_points_left)
            eyeball_pos_right = contouring(thresh[:, mid:], mid, img, end_points_right, True)
            print_eye_pos(eyeball_pos_left, eyeball_pos_right)
        # for (x, y) in shape[36:48]:
        #     cv2.circle(img, (x, y), 2, (255, 0, 0), -1)
    # show the image with the face detections + facial landmarks
    cv2.imshow('eyes', img)
    cv2.imshow("image", thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up


C:\Users\ADMIN\anaconda3\envs\env_rohan\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in long_scalars


Looking up


C:\Users\ADMIN\anaconda3\envs\env_rohan\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in long_scalars


Looking up
Looking up
Looking up
Looking up
Looking up
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking right
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking right
Looking left
Looking left
Looking left
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking right
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking left
Looking right
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Looking up
Lo